In [ ]:
from helper import export_projects, dataset_metadata, _export, download_datasets, upload_dataset, _export, _convert_dictkeys_to_snake
import datarobot as dr
import shutil
import pandas as pd
import os
import json
#Note1

In [ ]:
#Need to manually create this; because our codespace will NOT create an empty directory during init.
#Try to create a one-word .gitkeep and see if we can create this dir for them.
DIR = "migrations"
client = dr.Client()
me = client.get("account/info/").json()



## 1. 既存環境のプロジェクト・データセットをエキスポート

In [ ]:
#MissingSchemaエラーがあったら環境変数が未設定の可能性大
original_projects = export_projects()
original_catalog_metadata = dataset_metadata(original_projects)

In [ ]:
print("合計プロジェクト：", len(original_projects))
print("カタログIDのあるプロジェクト：", len([p for p in original_projects if p["catalogId"] is not None]))
print("-----------------------")

pd.DataFrame.from_records(original_projects)[["projectName","id","target","created"]]


## 2. 含みたいプロジェクトIDを以下のリストにて定義する

In [ ]:
# list_of_project_ids = ["66ac7fd29d62d870a9785e70","66ac813ddfabd5b3c254f0c7","66ab2bbb9b4a61619f66e5d2", "6699c1882f700a81438cae3f", "66a331cd6796f9292f7859aa","6699c1152f700a81438cadf2"]
list_of_project_ids = ["66ac7fd29d62d870a9785e70"]
proj_df = pd.DataFrame(original_projects)
filtered_df = proj_df.query("id in @list_of_project_ids")

In [ ]:
##移行対象アイテムのメタデータをファイルに書き出して保存場合

# with open(f"{DIR}/original-projects.json", "w") as f:
#     json.dump(original_projects, f, ensure_ascii=False, indent=4)
# original_catalog_metadata.to_csv(f"{DIR}/original-datasets.csv", index=False)

## 3. 次のバッチにフィルターして、Codespaceの空き容量を確認

In [ ]:
target_datasets = filtered_df.fileName.unique()
original_catalog_metadata.query("name in @target_datasets")

In [ ]:
#ストレージの空き容量は20GBあるため、１バッチあたり20GB以下にする必要ある

next_batch_df = original_catalog_metadata.query("name in @target_datasets")
print(next_batch_df["name"].values)
print(next_batch_df["datasetSize(MB)"].sum())

#### ストレージが次回ダウンロード分に対して十分な空き容量かの確認

In [ ]:
from IPython.core.display import HTML

raw = shutil.disk_usage(".")
free_mb = int(raw.free / (1_000_000))
print("空き容量: ", free_mb , "MB")

storage_ok = '<span style="color:green; font-weight:800">True</span>' if next_batch_df["datasetSize(MB)"].sum() < free_mb else '<span style="color:#f54c68; fontweight:800">False</span>'

print("次回移行分に必要な空き容量確保できる：：") 
HTML(storage_ok)


In [ ]:
downloaded_datasets = download_datasets(next_batch_df, DIR, 4)
#downloaded_datasets

## 5. Codespaceのストレージにダウンロードしたデータセットを新規環境にアップロード

In [ ]:
new_datasets = []
for cat in downloaded_datasets:
    record = {"name": cat["name"]}
    if cat["complete"]:
        resp = upload_dataset(cat, DIR=DIR)
        if "catalogId" in resp:
            record = record | resp
        new_datasets.append(record)

In [ ]:
new_datasets

## 6. 新しくアップロードしたデータセットのCatalogIDをプロジェクトデータに反映

In [ ]:
next_batch_projects = []
next_batch_datasets = list(next_batch_df.name)
target_projects = [p for p in original_projects if p["fileName"] in next_batch_datasets]

for idx, p in enumerate(target_projects):
    if p["stage"] != "modeling":
        continue
    matched_file = [ds for ds in new_datasets if ds["name"] == p["fileName"]]
    if len(matched_file) == 1:
        new_record = target_projects[idx].copy()
        new_record["catalogId"] = matched_file[0]["catalogId"]
        next_batch_projects.append(new_record)
print("次回処理予定のプロジェクト数: ", len(next_batch_projects))

#### Autopilot実行前にダウンロード
#### 完了するまで繰り返してprocessingStateを確認


In [ ]:
#新規環境に登録したデータセットのメタデータを取得
new_catalog = dataset_metadata(next_batch_projects,env="TARGET")
#データセット登録完了しているかを確認
new_catalog[["name","processingState"]]

## 7. Project / Autopilot 実行開始

In [ ]:
## RemoteDisconnectedやConnectionErrorが発生する場合がある。その場合再度実行して、正常に処理が始まります。

adv_option_filter = ["downsampledMinorityRows", "downsampledMajorityRows","responseCap"]
TARGET_WORKERS = 8

print("こちらにてプロジェクトの進捗をご確認ください。")
print(f'{os.environ.get("DATAROBOT_ENDPOINT")[:-7]}/manage-projects')
for proj in next_batch_projects:
    print("プロジェクト開始： ", proj["projectName"])
    dr.Context.use_case = None
    new_proj = dr.Project.create_from_dataset(
        dataset_id=proj["catalogId"],
        project_name=proj["projectName"],
        )
    
    if "datetimePartitionColumn" in proj["partition"]:
        partitioning_method = dr.DatetimePartitioningSpecification(
            datetime_partition_column=proj["partition"]["datetimePartitionColumn"],
        )
    elif proj["partition"]["cvMethod"] == "group":
        partitioning_method = dr.GroupCV(
            reps=proj["partition"]["reps"],
            holdout_pct=proj["paritition"]["holdoutPct"],
            partition_key_cols=proj["partition"]["partitionKeyCols"]
        )
    else:
        #特別な要件がある場合は調べておいてね！
        partitioning_method = None

        
    #TypeError: AdvancedOptions.__init__()のエラーがスローされたら該当属性をadv_option_filterに追加する
    adv_options = {k:v for (k,v) in proj["advancedOptions"].items() if k not in adv_option_filter}
    adv_options = _convert_dictkeys_to_snake(adv_options)
    if "primaryLocationColumn" in proj:
        adv_options["primary_location_column"] = proj["primaryLocationColumn"]
    advanced_options = dr.AdvancedOptions(**adv_options)
    
    try:
        new_proj = new_proj.analyze_and_model(
                    target=  proj["target"],
                    mode=    proj["autopilotMode"],
                    target_type=  proj["targetType"],
                    unsupervised_mode= proj["unsupervisedMode"],
                    unsupervised_type= proj["unsupervisedType"],
                    metric = proj["metric"],
                    positive_class = proj["positiveClass"],
                    advanced_options=advanced_options,                    	
            	    worker_count=TARGET_WORKERS,
                    partitioning_method=partitioning_method,
        )
    except dr.errors.ClientError as e:
        print("エラー発生：, ", e)

### 次回のバッチがある場合は移行済ファイルをStorageから削除した上

### ステップ2に戻り、次のバッチ範囲を決めて実行してください

In [ ]:
for file in next_batch_datasets:
    os.remove(f"{DIR}/{file}")


In [ ]:
exported_projects = export_projects(env="TARGET")

In [ ]:
#compare_projects = ["66ac844694d3910805cc1e5b","66ac8480347f45d126f97188"]
for p in exported_projects:
 #   if p["id"] in compare_projects:
        print(json.dumps(p, ensure_ascii=False))